Importanje potrebnih biblioteka.

In [26]:
import pandas as pd
import scipy.stats as stats
from collections import Counter

Definiranje raspona generacija i ispravno ocjenjivanje deepfake videa.

In [27]:
# gornji broj je exclusive
gen_z = range(15, 30)
gen_y = range(30, 45)
gen_x = range(45, 65)
gen_babyboom = range(65, 80)
gen_silent = range(80, 100)
not_gen_z = range(30, 100)

# 0 - autentičan, 1 - deepfake (3, 7, 9)
deepfake_correct = [0, 0, 1, 0, 0, 0, 1, 0, 1, 0]
deepfake_all_strings = [f'deepfake_{i}' for i in range (1, 11)]
deepfake_only_strings = ['deepfake_3', 'deepfake_7', 'deepfake_9']

Učitavanje dataseta.

In [28]:
dataset = pd.read_excel(r'dataset.xlsx')

"Ispravljanje" deepfake columna u 0 (ako ispitanik nije točno odredio video) ili 1 (ispitanik je točno prepoznao pravi/deepfake video). Dodavanje novog columna 'rezultat' gdje se nalazi ukupan rezultat ispitanika (raspon od 0 do 10). Dodavanje novog columna 'rezultat_deepfake' gdje se nalazi brroj ispravno prepoznatih deepfake videa (raspon od 0 do 3).

In [29]:
# odgovori za deepfake ispravljeni u bodove 0 ili 1 ovisno jesu li pogodili
corrected = dataset
for i, ds in enumerate(deepfake_all_strings):
    corrected[ds] = (corrected[ds] == deepfake_correct[i]).astype(int)

# dodan column rezultat sa zbrojem bodova za svakog ispitanika
corrected['rezultat'] = corrected[list(deepfake_all_strings)].sum(axis=1)

# dodan column rezultat_deepfake sa zbrojem ispravno prepoznatih videa koji su bili deepfake
corrected['rezultat_deepfake'] = corrected[list(deepfake_only_strings)].sum(axis=1)

df = pd.DataFrame(corrected, columns=['ispitanik', 'dob',
                                   'deepfake_1', 'deepfake_2', 'deepfake_3', 'deepfake_4', 'deepfake_5',
                                   'deepfake_6', 'deepfake_7', 'deepfake_8', 'deepfake_9', 'deepfake_10',
                                   'rezultat', 'rezultat_deepfake'], )
print(df)

     ispitanik  dob  deepfake_1  deepfake_2  deepfake_3  deepfake_4  \
0            1   48           1           1           1           0   
1            2   25           1           1           1           1   
2            3   23           1           0           1           0   
3            4   23           1           1           1           1   
4            5   23           1           0           1           1   
..         ...  ...         ...         ...         ...         ...   
97          98   69           1           1           0           1   
98          99   29           1           1           1           1   
99         100   64           1           1           0           1   
100        101   53           0           1           1           1   
101        102   37           1           1           1           1   

     deepfake_5  deepfake_6  deepfake_7  deepfake_8  deepfake_9  deepfake_10  \
0             1           0           1           1           1    

Računanje prosječnog rezultata ispitanika i određivanje granične p vrijednosti.

In [30]:
avg_result = corrected['rezultat'].mean()
print('Prosjek rezultata (max 10)', avg_result)
avg_result_depfake = corrected['rezultat_deepfake'].mean()
print('Prosjek rezultata ispravno prepoznatih deepfake videa (max 3)', avg_result_depfake)
p_value = 0.05
print('Kao granična p vrijednost se uzela', p_value)

Prosjek rezultata (max 10) 7.401960784313726
Prosjek rezultata ispravno prepoznatih deepfake videa (max 3) 2.372549019607843
Kao granična p vrijednost se uzela 0.05


Prvi test:
H0 - Sve generacije jednako dobro raspoznaju deepfake videe
H1 - Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije

In [31]:
t1h0 = 'Sve generacije jednako dobro raspoznaju deepfake videe'
t1h1 = 'Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije'
print('H0:', t1h0)
print('H1:', t1h1)
print()

not_gen_z_avg = corrected[corrected['dob'].isin(not_gen_z)]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji nisu dio gen Z generacije:', not_gen_z_avg)
gen_z_data = corrected[corrected['dob'].isin(gen_z)]['rezultat']
print('Prosječni rezultat ispitanika koji su dio gen Z generacije:', gen_z_data.mean())
_, p_value_test = stats.ttest_1samp(gen_z_data, not_gen_z_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')

H0: Sve generacije jednako dobro raspoznaju deepfake videe
H1: Generacija Z bolje raspoznaje deepfake u odnosu na ostale generacije

Prosječni rezultat ispitanika koji nisu dio gen Z generacije: 7.083333333333333
Prosječni rezultat ispitanika koji su dio gen Z generacije: 7.685185185185185
P vrijednost je 0.014324984472095098
Odbacuje se H0.


Drugi test:
H0 - Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže
H1 - Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe

In [32]:
t2h0 = 'Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže'
t2h1 = 'Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe'
print('H0:', t2h0)
print('H1:', t2h1)
print()

not_social_media_avg = corrected[corrected['cesto_mreze'] > 1]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji rijetko prate društvene mreže:', not_social_media_avg)
social_media_data = corrected[corrected['cesto_mreze'] == 1]['rezultat']
print('Prosječni rezultat ispitanika koji često prate društvene mreže:', social_media_data.mean())
_, p_value_test = stats.ttest_1samp(social_media_data, not_social_media_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')

H0: Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko prate društvene mreže
H1: Ispitanici koji društvene mreže prate barem jednom dnevno bolje prepoznaju deepfake videe

Prosječni rezultat ispitanika koji rijetko prate društvene mreže: 6.7368421052631575
Prosječni rezultat ispitanika koji često prate društvene mreže: 7.554216867469879
P vrijednost je 2.0550580600501343e-05
Odbacuje se H0.


Treći test:
H0 - Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije
H1 - Ispitanici koji uopće nemaju povjerenja u medije ne raspoznaju dobro deepfake videe

In [33]:
t3h0 = 'Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije'
t3h1 = 'Ispitanici koji uopće nemaju povjerenja u medije ne raspozaju dobro deepfake videe'
print('H0:', t3h0)
print('H1:', t3h1)
print()

# povjerenje u tv
trust_tv_avg = corrected[corrected['povjerenje_tv'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju televiziji:', trust_tv_avg)
not_trust_tv_data = corrected[corrected['povjerenje_tv'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju televiziji:', not_trust_tv_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_tv_data, trust_tv_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

# povjerenje u radio
trust_radio_avg = corrected[corrected['povjerenje_radio'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju radiju:', trust_radio_avg)
not_trust_radio_data = corrected[corrected['povjerenje_radio'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju radiju:', not_trust_radio_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_radio_data, trust_radio_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

#povjerenje u novine
trust_news_avg = corrected[corrected['povjerenje_novine'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju novinama:', trust_news_avg)
not_trust_news_data = corrected[corrected['povjerenje_novine'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju novinama:', not_trust_news_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_news_data, trust_news_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

#povjerenje u portale
trust_portal_avg = corrected[corrected['povjerenje_portali'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju portalima:', trust_portal_avg)
not_trust_portal_data = corrected[corrected['povjerenje_portali'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju portalima:', not_trust_portal_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_portal_data, trust_portal_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

# povjerenje u društvene mreže
trust_sm_avg = corrected[corrected['povjerenje_mreze'] <= 3]['rezultat'].mean()
print('Prosječni rezultat ispitanika koji vjeruju društvenim mrežama:', trust_sm_avg)
not_trust_sm_data = corrected[corrected['povjerenje_mreze'] > 3]['rezultat']
print('Prosječni rezultat ispitanika koji ne vjeruju društvenim mrežama:', not_trust_sm_data.mean())
_, p_value_test = stats.ttest_1samp(not_trust_sm_data, trust_sm_avg)

print('P vrijednost je', p_value_test)
if p_value_test < p_value:
    print('Odbacuje se H0.')
else:
    print('Nije moguće odbaciti H0.')
print()

H0: Svi ispitanici podjednako dobro raspoznaju deepfake videe neovisno o tome koliko imaju povjerenja u medije
H1: Ispitanici koji uopće nemaju povjerenja u medije ne raspozaju dobro deepfake videe

Prosječni rezultat ispitanika koji vjeruju televiziji: 7.4605263157894735
Prosječni rezultat ispitanika koji ne vjeruju televiziji: 7.230769230769231
P vrijednost je 0.5251477006498033
Nije moguće odbaciti H0.

Prosječni rezultat ispitanika koji vjeruju radiju: 7.512820512820513
Prosječni rezultat ispitanika koji ne vjeruju radiju: 7.041666666666667
P vrijednost je 0.26271987675661923
Nije moguće odbaciti H0.

Prosječni rezultat ispitanika koji vjeruju novinama: 7.506329113924051
Prosječni rezultat ispitanika koji ne vjeruju novinama: 7.043478260869565
P vrijednost je 0.24794753970701763
Nije moguće odbaciti H0.

Prosječni rezultat ispitanika koji vjeruju portalima: 7.491803278688525
Prosječni rezultat ispitanika koji ne vjeruju portalima: 7.2682926829268295
P vrijednost je 0.45516834085782

Profil ispitanika koji ispravno ne prepoznaju deepfake videe i označe ih kako autentične.

In [44]:
correct_3 = corrected['rezultat_deepfake'].value_counts()[3]
print('Broj ispitanika koji su ispravno prepoznali sve deepfake videe:', correct_3)
correct_2 = corrected['rezultat_deepfake'].value_counts()[2]
print('Broj ispitanika koji su ispravno prepoznali 2 od 3 deepfake videa:', correct_2)
correct_1 = corrected['rezultat_deepfake'].value_counts()[1]
print('Broj ispitanika koji su ispravno prepoznali 1 od 2 deepfake videa:', correct_1)
correct_0 = corrected['rezultat_deepfake'].value_counts()[0]
print('Broj ispitanika koji nisu ispravno prepoznali nijedan deepfake video:', correct_0)
print()

threshold_not_correct = 1

correct_low = corrected[corrected['rezultat_deepfake'] <= threshold_not_correct]
correct_low_lines = correct_low.shape[0]
all_lines = corrected.shape[0]
# correct_high = corrected[corrected['rezultat_deepfake'] > threshold_not_correct]

# spol
print('Najčešći spol:')
for item, value in Counter(corrected['spol']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešći spol koji ne raspoznaje:')
for item, value in Counter(correct_low['spol']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# obrazovanje
print('Najčešće obrazovanje:')
for item, value in Counter(corrected['max_obrazovanje']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće obrazovanje koje ne raspoznaje:')
for item, value in Counter(correct_low['max_obrazovanje']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# prebivalište
print('Najčešće prebivalište:')
for item, value in Counter(corrected['prebivaliste']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće prebivalište koje ne raspoznaje:')
for item, value in Counter(correct_low['prebivaliste']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()


# povjerenje tv
print('Najčešće povjerenje tv:')
for item, value in Counter(corrected['povjerenje_tv']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće povjerenje tv koje ne raspoznaje:')
for item, value in Counter(correct_low['povjerenje_tv']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# povjerenje radio
print('Najčešće povjerenje radio:')
for item, value in Counter(corrected['povjerenje_radio']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće povjerenje radio koje ne raspoznaje:')
for item, value in Counter(correct_low['povjerenje_radio']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# povjerenje novine
print('Najčešće povjerenje novine:')
for item, value in Counter(corrected['povjerenje_novine']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće povjerenje novine koje ne raspoznaje:')
for item, value in Counter(correct_low['povjerenje_novine']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# povjerenje portali
print('Najčešće povjerenje portali:')
for item, value in Counter(corrected['povjerenje_portali']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće povjerenje portali koje ne raspoznaje:')
for item, value in Counter(correct_low['povjerenje_portali']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# povjerenje mreze
print('Najčešće povjerenje mreže:')
for item, value in Counter(corrected['povjerenje_mreze']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće povjerenje mreže koje ne raspoznaje:')
for item, value in Counter(correct_low['povjerenje_mreze']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

# povjerenje poruke
print('Najčešće povjerenje poruke:')
for item, value in Counter(corrected['povjerenje_poruke']).items():
    print(f'{item}: {value*100/all_lines}%')
print('Najčešće povjerenje poruke koje ne raspoznaje:')
for item, value in Counter(correct_low['povjerenje_poruke']).items():
    print(f'{item}: {value*100/correct_low_lines}%')
print()

Broj ispitanika koji su ispravno prepoznali sve deepfake videe: 56
Broj ispitanika koji su ispravno prepoznali 2 od 3 deepfake videa: 30
Broj ispitanika koji su ispravno prepoznali 1 od 2 deepfake videa: 14
Broj ispitanika koji nisu ispravno prepoznali nijedan deepfake video: 2

Najčešći spol:
0: 69.6078431372549%
1: 30.392156862745097%
Najčešći spol koji ne raspoznaje:
0: 75.0%
1: 25.0%

Najčešće obrazovanje:
4: 32.35294117647059%
3: 32.35294117647059%
2: 33.333333333333336%
1: 1.9607843137254901%
Najčešće obrazovanje koje ne raspoznaje:
4: 50.0%
3: 18.75%
2: 31.25%

Najčešće prebivalište:
1: 52.94117647058823%
2: 26.470588235294116%
3: 20.58823529411765%
Najčešće prebivalište koje ne raspoznaje:
2: 18.75%
1: 68.75%
3: 12.5%

Najčešće povjerenje tv:
3: 44.11764705882353%
4: 18.627450980392158%
2: 25.49019607843137%
1: 4.901960784313726%
5: 6.862745098039215%
Najčešće povjerenje tv koje ne raspoznaje:
3: 31.25%
1: 18.75%
4: 18.75%
2: 31.25%

Najčešće povjerenje radio:
3: 48.03921568627